In [36]:
!pip -q install openai

In [37]:
import openai
import json


In [38]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [39]:
def summarize_forecast(message, function_name, function_response, city):
    summary = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": f"What is the weather like in {city}?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
    return summary["choices"][0]["message"]

In [40]:
city = "pakistan"

def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": f"What is the weather like in {city}?"}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]

    if message.get("function_call"):
        function_name = message["function_call"]["name"]

        if function_name == "get_current_weather": 
            function_response = get_current_weather(
                location=message.get("location"),
                unit=message.get("unit"),
            )

            summary = summarize_forecast(message, function_name, function_response, city)

            return summary

print(run_conversation())

{
  "role": "assistant",
  "content": "The current weather in Karachi, Pakistan is sunny and windy with a temperature of 72 degrees. Please note that weather conditions can vary in different parts of Pakistan."
}
